# Prédire la durée de validite de produit 

In [1]:
import pyodbc
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier


# Connexion SQL Server
server = 'DESKTOP-OQ959SK'
database = 'FinalDW'
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

try:
    conn = pyodbc.connect(conn_str)
    print("✅ Connexion réussie !")
except Exception as e:
    print("❌ Échec de connexion :", e)

# Requête SQL
query = """
SELECT 
    dp.ProductID,
    dp.ProductName,
    dc.Category AS Category_Name,
    db.brandname AS Brand_Name,
    COUNT(DISTINCT dm.Material_ID) AS Nb_Ingredients,
    AVG(fd.Dosage) AS Dosage_Moyen,
    dd_m.Year AS Manufacture_Year,
    dd_m.Month AS Manufacture_Month,
    dd_e.Year AS Expiry_Year,
    dd_e.Month AS Expiry_Month,
    DATEDIFF(DAY, dd_m.Full_Date, dd_e.Full_Date) AS Duree_Validite_Jours

FROM Fact_Production fp
JOIN Dim_Product dp ON fp.Product_PK = dp.Product_PK
JOIN Dim_Category dc ON dp.Category_FK = dc.Category_PK
JOIN Dim_Brand db ON dp.Brand_FK = db.Brand_PK
JOIN Fact_Dosage fd ON fd.Prod_FK = dp.Product_PK
JOIN Dim_Material dm ON fd.Material_FK = dm.Material_PK
JOIN Dim_Date dd_m ON fp.ManufactureDate_PK = dd_m.Date_PK
JOIN Dim_Date dd_e ON fp.ExpiryDate_PK = dd_e.Date_PK

GROUP BY 
    dp.ProductID,
    dp.ProductName,
    dc.Category,
    db.brandname,
    dd_m.Year, dd_m.Month,
    dd_e.Year, dd_e.Month,
    dd_m.Full_Date,
    dd_e.Full_Date
"""

# Lire les résultats dans un DataFrame
df = pd.read_sql(query, conn)
df.head()
print(df)

✅ Connexion réussie !


C:\Users\Mariem\AppData\Local\Temp\ipykernel_12360\2279726980.py:59: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


   ProductID   ProductName Category_Name                 Brand_Name  \
0     PR-001  Hair Serum 1      Haircare               walker group   
1     PR-002   Body Wash 2      Bodycare      lyons zuniga and wolf   
2     PR-006   Face Wash 6      Skincare  brown townsend and dalton   
3     PR-007       Blush 7        Makeup    gibbs medina and carter   
4     PR-010    Mascara 10        Makeup   jacobs obrien and davila   
..       ...           ...           ...                        ...   
76    PR-875     Blush 538        Makeup   hansen hopkins and smith   
77    PR-883  Lipstick 542        Makeup             lewis and sons   
78    PR-917   Mascara 559        Makeup      lyons zuniga and wolf   
79    PR-919     Blush 560        Makeup   hansen hopkins and smith   
80    PR-993     Blush 597        Makeup               walker group   

    Nb_Ingredients  Dosage_Moyen Manufacture_Year Manufacture_Month  \
0                2             8             2024                08   
1    

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Variables
categorical_features = ['ProductName', 'Category_Name', 'Brand_Name']
numerical_features = [
    'Nb_Ingredients',
    'Dosage_Moyen',
    'Manufacture_Year',
    'Manufacture_Month',
    'Expiry_Year',
    'Expiry_Month'
]

# Preprocessing
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(transformers=[
    ('cat', categorical_transformer, categorical_features),
    ('num', numerical_transformer, numerical_features)
])

# Split train / test
from sklearn.model_selection import train_test_split
print(X.columns.tolist())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Models
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

# Train and evaluate each model
for name, model in models.items():
    pipeline = Pipeline([
        ('preprocessing', preprocessor),
        ('model', model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    print(f"\n🔹 {name}")
    print("MAE:", round(mean_absolute_error(y_test, y_pred), 2))
    print("RMSE:", round(np.sqrt(mean_squared_error(y_test, y_pred)), 2))
    print("R²:", round(r2_score(y_test, y_pred), 2))


['ProductName', 'Category_Name', 'Brand_Name', 'Nb_Ingredients', 'Dosage_Moyen', 'Manufacture_Year', 'Manufacture_Month', 'Expiry_Year', 'Expiry_Month']

🔹 Linear Regression
MAE: 11.23
RMSE: 14.75
R²: 0.99

🔹 Random Forest
MAE: 23.48
RMSE: 30.03
R²: 0.96

🔹 XGBoost
MAE: 20.47
RMSE: 31.71
R²: 0.95
